In [1]:
#import libraries

import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup as soup

In [2]:
# Set URL as a variable
url = 'https://nces.ed.gov/programs/digest/d95/dtab077.asp'

# Use requests.post becuase it'll pull data after the webpage has loaded
r=requests.post(url)

# create a "soup" object 
data = soup(r.text, 'html.parser')

In [3]:
# Identify element tag with target data
data.select('table td pre')[0]

<pre style="text-align:left; padding-left:4px">____________________________________________________________________________________________________________________________________________________________
                      |                                                                   |                                                      | Percent
                      |                    Current dollars                                |                 Constant 1994-95 dollars\1\          | change,
                      |                                                                   |                                                      |1979-80 to
        State         |___________________________________________________________________|______________________________________________________|1994-95 in
                      |       |        |        |        |        |           |           |        |        |        |        |        |         |constant
                 

In [4]:
# Our data of interest is the text that is formatted like a table
# Select that data and return as text
doi= data.select('table td pre')[0].text
print(doi)

____________________________________________________________________________________________________________________________________________________________
                      |                                                                   |                                                      | Percent
                      |                    Current dollars                                |                 Constant 1994-95 dollars\1\          | change,
                      |                                                                   |                                                      |1979-80 to
        State         |___________________________________________________________________|______________________________________________________|1994-95 in
                      |       |        |        |        |        |           |           |        |        |        |        |        |         |constant
                      |1969-70|1979-80 |1989-90 |1991-92 |1992-9

In [5]:
# read_csv was taking in object as a file name
# trouble shooting said to use io.StringIO
#  

import io

salary_df = pd.read_csv(io.StringIO(doi),sep='|', #separate columns by |-symbol
            skiprows=6,
            usecols=list(range(0,8))
            )
salary_df

,,1969-70,1979-80,1989-90,1991-92,1992-93,1993-94,1994-95
0,______________________,_______,________,________,________,________,___________,___________
1,1,2,3,4,5,6,7,8
2,______________________,_______,________,________,________,________,___________,___________
3,United States .....,"$8,626","$15,970","$31,367","$34,063","$35,029","\2\$35,819","\2\$36,933"
4,,_______,________,________,________,________,___________,___________
...,...,...,...,...,...,...,...,...
61,Washington ...........,"9,225","18,820","30,457","34,823","35,759","35,863","36,120"
62,West Virginia.........,"7,650","13,710","22,842","27,366","30,301","30,549","31,923"
63,Wisconsin ............,"8,963","16,006","31,921","35,227","35,926","35,990","\2\ 37,349"
64,Wyoming ..............,"8,232","16,012","28,141","30,425","30,080","30,952","31,300"


### Clean the df

In [6]:
# Rename first column
salary_df.rename(columns={salary_df.columns[0]:'State'}, inplace=True)

salary_df.columns

Index(['State', '1969-70', '1979-80 ', '1989-90 ', '1991-92 ', '1992-93 ',
       '   1993-94 ', '   1994-95 '],
      dtype='object')

In [7]:
# Remove white space from other column names

# Create an empty dicitonary
new_column_names = {}

# Loop thru column names and remove white space
for i in list(range(len(salary_df.columns))):
    new_column_names[salary_df.columns[i]] = salary_df.columns[i].strip()

salary_df.rename(columns=new_column_names, inplace=True)

salary_df.head()

,State,1969-70,1979-80,1989-90,1991-92,1992-93,1993-94,1994-95
0,______________________,_______,________,________,________,________,___________,___________
1,1,2,3,4,5,6,7,8
2,______________________,_______,________,________,________,________,___________,___________
3,United States .....,"$8,626","$15,970","$31,367","$34,063","$35,029","\2\$35,819","\2\$36,933"
4,,_______,________,________,________,________,___________,___________


In [8]:
# remove '...' from state name column and then set as index
def period_remover(value):
    
    temp = value.strip()
    
    if len(temp) > 1 and temp[0].lower() in ('abcdefghijklmnopqrstuvwxyz'):
        return temp.split('.')[0].strip()
    else:
        return np.NaN

salary_df_clean = salary_df.copy()

salary_df_clean['State'] = salary_df_clean['State'].apply(period_remover)

salary_df_clean

,State,1969-70,1979-80,1989-90,1991-92,1992-93,1993-94,1994-95
0,NaN,_______,________,________,________,________,___________,___________
1,NaN,2,3,4,5,6,7,8
2,NaN,_______,________,________,________,________,___________,___________
3,United States,"$8,626","$15,970","$31,367","$34,063","$35,029","\2\$35,819","\2\$36,933"
4,NaN,_______,________,________,________,________,___________,___________
...,...,...,...,...,...,...,...,...
61,Washington,"9,225","18,820","30,457","34,823","35,759","35,863","36,120"
62,West Virginia,"7,650","13,710","22,842","27,366","30,301","30,549","31,923"
63,Wisconsin,"8,963","16,006","31,921","35,227","35,926","35,990","\2\ 37,349"
64,Wyoming,"8,232","16,012","28,141","30,425","30,080","30,952","31,300"


In [9]:
cleaner_salary_df = salary_df_clean.dropna(axis=0)
cleaner_salary_df.head(10)

,State,1969-70,1979-80,1989-90,1991-92,1992-93,1993-94,1994-95
3,United States,"$8,626","$15,970","$31,367","$34,063","$35,029","\2\$35,819","\2\$36,933"
5,Alabama,"6,818","13,060","24,828","26,971","26,953","28,705","31,144"
6,Alaska,"10,560","27,210","43,153","44,661","46,701","47,512","47,951"
7,Arizona,"8,711","15,054","29,402","31,176","31,352","31,800","\2\ 32,090"
8,Arkansas,"6,307","12,299","22,352","27,070","27,433","28,098","28,409"
9,California,"10,315","18,020","37,998","39,922","40,035","40,264","\2\ 40,667"
11,Colorado,"7,761","16,205","30,758","33,072","33,541","33,826","34,571"
12,Connecticut,"9,262","16,229","40,461","46,971","48,343","49,769","51,300"
13,Delaware,"9,015","16,148","33,377","34,548","36,217","37,469","39,076"
14,District of Columbia,"10,285","22,190","38,402","38,798","38,702","42,543","42,959"


In [10]:
def data_to_int(val):
    return int(''.join(re.findall(r'\d',val.split('\\')[-1])))

In [11]:
data_to_int('11,001')

11001

In [12]:
salary_df_final = cleaner_salary_df.copy()

for i in list(range(len(salary_df_final.columns)-1)):
    salary_df_final[salary_df_final.columns[i+1]] = salary_df_final[salary_df_final.columns[i+1]].apply(data_to_int)

In [13]:
salary_df_final

,State,1969-70,1979-80,1989-90,1991-92,1992-93,1993-94,1994-95
3,United States,8626,15970,31367,34063,35029,35819,36933
5,Alabama,6818,13060,24828,26971,26953,28705,31144
6,Alaska,10560,27210,43153,44661,46701,47512,47951
7,Arizona,8711,15054,29402,31176,31352,31800,32090
8,Arkansas,6307,12299,22352,27070,27433,28098,28409
9,California,10315,18020,37998,39922,40035,40264,40667
11,Colorado,7761,16205,30758,33072,33541,33826,34571
12,Connecticut,9262,16229,40461,46971,48343,49769,51300
13,Delaware,9015,16148,33377,34548,36217,37469,39076
14,District of Columbia,10285,22190,38402,38798,38702,42543,42959


In [14]:
salary_df_final.to_csv('Data/1995_avg_teacher_salaries.csv', index=False)

In [ ]:
# 2006 and on can use pd.read_html 

# For everything before that: pd.read_csv

# 1995 has a unique url ('https://nces.ed.gov/programs/digest/d95/dtab077.asp')

# Table 77 for 1996 ('https://nces.ed.gov/programs/digest/d96/d96t077.asp')
# Table 77 for 2005 ('https://nces.ed.gov/programs/digest/d05/tables/dt05_077.asp')

# Table 78 for 1997 ('https://nces.ed.gov/programs/digest/d97/d97t078.asp')
# Table 78 for 2000-04 ('https://nces.ed.gov/programs/digest/d00/dt078.asp')

#Table 79 in 1998,99 ('https://nces.ed.gov/programs/digest/d98/d98t079.asp')

for year in range(1995,2013):
    print(year)
    try:
        url = f'https://nces.ed.gov/programs/digest/d{str(year)[2:]}/d{str(year)[2:]}t077.asp'
        r=requests.post(url)
        data = soup(r.text, 'html.parser')

        table_list = data.select('table td pre')

        print(len(table_list))   
    except:
        print(f'No data in this link: {url}')

In [352]:
#import libraries

import io
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup as soup

def table_scraper(url):
    # ----------------------------------------------------------------------------------------------------------------------
    
    # Use requests.post becuase it'll pull data after the webpage has loaded
    r=requests.post(url)

    # create a "soup" object 
    data = soup(r.text, 'html.parser')

    # ----------------------------------------------------------------------------------------------------------------------

    # Our data of interest is the text that is formatted like a table
    # Select that data and return as text
    doi= data.select('table td pre')[0].text

    # ----------------------------------------------------------------------------------------------------------------------

    salary_df = pd.read_csv(io.StringIO(doi),sep='|', #separate columns by |-symbol
                skiprows=6,
                usecols=list(range(0,8))
                )

    # ----------------------------------------------------------------------------------------------------------------------

    # Rename first column
    salary_df.rename(columns={salary_df.columns[0]:'State'}, inplace=True)

    # ----------------------------------------------------------------------------------------------------------------------

    # Remove white space from other column names

    # Create an empty dicitonary
    new_column_names = {}

    # Loop thru column names and remove white space
    for i in list(range(len(salary_df.columns))):
        new_column_names[salary_df.columns[i]] = salary_df.columns[i].strip()

    salary_df.rename(columns=new_column_names, inplace=True)

    # ----------------------------------------------------------------------------------------------------------------------

    # remove '...' from state name column and then set as index
    def period_remover(value):
        
        temp = value.strip()
        
        if len(temp) > 1 and temp[0].lower() in ('abcdefghijklmnopqrstuvwxyz'):
            return temp.split('.')[0]
        else:
            return np.NaN

    salary_df_clean = salary_df.copy()

    salary_df_clean['State'] = salary_df_clean['State'].apply(period_remover)

    # ----------------------------------------------------------------------------------------------------------------------

    cleaner_salary_df = salary_df_clean.dropna(axis=0)

    # ----------------------------------------------------------------------------------------------------------------------

    def data_to_int(val):
        return int(''.join(re.findall(r'\d',val.split('\\')[-1])))

    salary_df_final = cleaner_salary_df.copy()

    for i in list(range(len(salary_df_final.columns)-1)):
        salary_df_final[salary_df_final.columns[i+1]] = salary_df_final[salary_df_final.columns[i+1]].apply(data_to_int)

    return salary_df_final